In [1]:
!pip install gym[all]
!pip install box2d-py
!apt-get install python-opengl -y
!apt install xvfb -y

# Special gym environment
#!pip install gym[atari]

     |████████████████████████████████| 450kB 9.7MB/s 
     |████████████████████████████████| 4.0MB 43.0MB/s 
  Created wheel for glfw: filename=glfw-1.8.4-cp36-none-any.whl size=23901 sha256=fc274ae9460638743d6c2cf0d24a3dd439c253ad375c6aea51b2b75b2b8a2453
  Stored in directory: /root/.cache/pip/wheels/e3/d8/6e/0d188598f646bff34b79eab04d5203d3649248fbe23253c63f
Successfully built glfw
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 0s (3,780 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 132684 files and directories currently instal

In [0]:
import tensorflow as tf
import numpy as np
import gym
import math
#import tensorflow_probability as tfp
#display gym

tf.compat.v1.disable_eager_execution()
""""""
print(tf.__version__)


class ppo():
	def __init__(self,name,s_dim,a_dim,memory,a_bound):
		self.s_dim = s_dim
		self.a_dim =a_dim
		self.memory = memory
		self.a_bound =a_bound
		self.name = name
		self.policy  = self.make_policy()
		self.critic = self.make_critic()
		
	def make_policy(self):
			

		state_inputs = tf.keras.Input(shape=(self.s_dim,), name='state')
		advantage = tf.keras.Input(shape=(1, ), name="Advantage")
		action= tf.keras.Input(shape=(self.a_dim,), name="action")
		x = tf.keras.layers.Dense(16, activation='relu')(state_inputs)
		x1 = tf.keras.layers.Dense(16, activation='relu')(x)
		mu_0 = tf.keras.layers.Dense(self.a_dim, activation='tanh')(x1)
		x2 = tf.keras.layers.Dense(16, activation='relu')(x)
		sigma_0 = tf.keras.layers.Dense(self.a_dim, activation='relu')(x2)

		mu = tf.keras.layers.Lambda(lambda x:tf.clip_by_value(x * self.a_bound,-self.a_bound ,self.a_bound) )(mu_0)
		#mu = tf.keras.layers.Reshape((1,self.a_dim))(mu)

		covari = tf.keras.layers.Lambda(lambda x:tf.clip_by_value(x,1e-2 , 1e+02))(sigma_0)
		#covari = tf.keras.layers.Reshape((1,self.a_dim))(covari)
		
		#concat=custom_concat()
		mucov=tf.keras.layers.concatenate([mu,covari],axis=-1)
		#mucov =  tf.keras.layers.Lambda(lambda x:tf.keras.backend.expand_dims(x,1))(mucov)
		
		#mucov=concat([mu,covari])

		def proximal_policy_optimization_loss(advantage, action):
			loss_clipping = 0.2
			entropy_loss = 0.0
			pi=3.1415926
			k=self.a_dim

			def loss(y_true,y_pred):
				
				mu =y_pred[:,:k]#batch,k
				cov = y_pred[:,k:]#batch,k
				old_mu = y_true[:,:k]
				old_cov = y_true[:,k:]
				x= action

				det = tf.keras.backend.prod(cov,axis=1,keepdims=True)
				inv = 1/cov#tf.linalg.inv(cov_mat)
				norm_const = 1.0/ ( tf.keras.backend.pow(2*pi,k/2) * tf.keras.backend.pow(det,1.0/2) )
				x_mu = x - mu
				#tf.print("deter",det)
				#tf.print(" cov inv_cov",cov,inv)
				#tf.print("x,mu,xmu",x,mu,x_mu)
				x_mu_sq = tf.keras.backend.square(x_mu)	
				prod  = inv*x_mu_sq
				#tf.print('x_mu_sq',x_mu_sq)
				#tf.print('prod inv*x_mu_sq',prod,tf.keras.backend.int_shape(prod))

				prod2 =tf.keras.backend.sum(prod,axis=1,keepdims=True) 
				#tf.print("prod2 elemt sum ",prod2,tf.keras.backend.int_shape(prod2))
				result = tf.keras.backend.exp( -0.5 *prod2)
				pdf = norm_const * result
				
				old_det = tf.keras.backend.prod(old_cov,axis=1,keepdims=True)
				##tf.print("old deter",old_det)
				old_inv = 1/old_cov#tf.linalg.old_inv(old_cov_mat)
				old_norm_const = 1.0/ ( tf.keras.backend.pow(2*pi,k/2) * tf.keras.backend.pow(old_det,1.0/2) )
				old_x_mu = x - old_mu
				#tf.print("old_cov old_inv_old_cov",old_cov,old_inv)
				#tf.print("x,old_mu,xold_mu",x,old_mu,old_x_mu)
				old_x_mu_sq = tf.keras.backend.square(old_x_mu)	
				old_prod  = old_inv*old_x_mu_sq
				#tf.print('old_x_mu_sq',old_x_mu_sq)
				#tf.print('old_prod oldinv*oldx_mu_sq',old_prod,tf.keras.backend.int_shape(old_prod))

				old_prod2 =tf.keras.backend.sum(old_prod,axis=1,keepdims=True) 
				#print("old_prod2 elemt sum ",old_prod2,tf.keras.backend.int_shape(old_prod2))
				old_result = tf.keras.backend.exp( -0.5 *old_prod2)
				old_pdf = old_norm_const * old_result					

					

				log_pdf = tf.keras.backend.log(pdf + tf.keras.backend.epsilon())
				
				old_log_pdf = tf.keras.backend.log(old_pdf + tf.keras.backend.epsilon() )
				entropy =  tf.keras.backend.sum(0.5 * (tf.keras.backend.log(2. * pi * det) + 1.))

				
				r = tf.keras.backend.exp(log_pdf- old_log_pdf)
				loss = -tf.keras.backend.mean(tf.keras.backend.minimum(r * advantage, tf.keras.backend.clip(r, min_value=1 - loss_clipping,max_value=1 + loss_clipping) * advantage)) #+ entropy_loss *entropy
				
				return loss
			return loss	
		policy= tf.keras.Model(inputs=(state_inputs, advantage,action), outputs=mucov, name='p_actor_model')
		policy.compile(loss=proximal_policy_optimization_loss(advantage=advantage,action=action), optimizer=tf.keras.optimizers.Adam(lr=0.0001))
		return policy

	def make_critic(self):
		state_inputs = tf.keras.Input(shape=(self.s_dim,), name='state')

		x = tf.keras.layers.Dense(16, activation='relu')(state_inputs)
		x = tf.keras.layers.Dense(16, activation='relu')(x)
		value_outputs = tf.keras.layers.Dense(1, activation=None)(x)
		critic= tf.keras.Model(inputs=state_inputs, outputs=value_outputs, name='p_critic_model')
		critic.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['accuracy'])
		return critic
	def save_weights(self):
		actorpath=r"C:\\Users\\Dell\\Desktop\\this_year\\SCM-RL\\ppo_continous_"+self.name+"actor.h5"
		criticpath=r"C:\\Users\\Dell\\Desktop\\this_year\\SCM-RL\\ppo_continous_"+self.name+"critic.h5"
		self.policy.save_weights(actorpath)
		self.critic.save_weights(criticpath)
		print("saved")
	def load_weights(self):
		actorpath=r"C:\\Users\\Dell\\Desktop\\this_year\\SCM-RL\\ppo_continous_"+self.name+"actor.h5"
		criticpath=r"C:\\Users\\Dell\\Desktop\\this_year\\SCM-RL\\ppo_continous_"+self.name+"critic.h5"
		self.policy.load_weights(actorpath)
		self.critic.load_weights(criticpath)
		print("loaded")

	def gae_calc(self,val,val_,rew,done):
		mask=1 
		gae=0
		gamma=0.95
		lambd = 0.95
		returns=np.zeros_like(val)
		for i in reversed(range(0,len(val))):
			mask=1
			if done[i]:
				mask = 0 	
			delta=rew[i]+gamma*val_[i]*mask - val[i]
			gae=delta+gamma*lambd*mask*gae
			returns[i]=gae+val[i]
		return returns
		
	def adv_calc(self,val,val_,rew,done):
		gamma=0.99
		returns=np.zeros_like(val)
		for i in range(0,len(val)):
			returns[i] = rew[i] + (1- done[i])*val_[i]*gamma
		return returns
	def train(self,batch=512,epochs=10):
		obs =np.array( self.memory.batch_s)
		values = self.critic.predict(np.array(self.memory.batch_s))
		values_ = self.critic.predict(np.array(self.memory.batch_s_))
		returns = self.adv_calc(values,values_,self.memory.batch_r,self.memory.batch_done)	
		advantage=returns-values
		Action=np.array(self.memory.batch_a)
		Old_Prediction_musig =np.array(self.memory.musig) 
		Old_cov = np.array(self.memory.cov)
		#print("policy_train")
		self.policy.fit(x=(obs,advantage,Action),y=Old_Prediction_musig,batch_size=batch,shuffle=True, epochs=epochs, verbose=False)
		#print("value_train")
		self.critic.fit([obs],[returns], batch_size=batch, shuffle=True, epochs=epochs, verbose=False)
		self.memory.clear()




class Memory:
	def __init__(self):
		self.batch_s = []
		self.batch_a = []
		self.batch_r = []
		self.batch_s_ = []
		self.batch_done = []
		self.musig = []
		self.cov=[]
	def store(self, s, a, s_, r, done,musig,cov):
		self.batch_s.append(s)
		self.batch_a.append(a)
		self.batch_r.append(r)
		self.batch_s_.append(s_)
		self.batch_done.append(done)
		self.musig.append(musig)
		self.cov.append(cov)
	def clear(self):
		self.batch_s.clear()
		self.batch_a.clear()
		self.batch_r.clear()
		self.batch_s_.clear()
		self.batch_done.clear()
		self.musig.clear()
		self.cov.clear()
	def cnt_samples(self):
		return len(self.batch_s)


envs = ['Pendulum-v0','MountainCarContinuous-v0','LunarLanderContinuous-v2','BipedalWalker-v2','BipedalWalkerHardcore-v2']
env= envs[2]
#env1=gym.make('Pendulum-v0')
env1=gym.make(env)
env1=env1.unwrapped

s_dim1 = env1.observation_space.shape[0]
print(s_dim1)
a_dim1 =env1.action_space.shape[0]
print(a_dim1)
a_bound1 = env1.action_space.high[0]
print(a_bound1)
DUMMY_ACTION1, DUMMY_VALUE1 = np.zeros((1,a_dim1)), np.zeros((1, 1))

memory_1=Memory()
agent_1 =  ppo(name = "ppo_agent_01",s_dim=s_dim1 ,a_dim= a_dim1,memory = memory_1,a_bound=a_bound1)








episodes = 20000000
steps = 1000
render=0
var=1# need decaying exploration noise to solve mountain car


for episode in range(1,episodes):
	s1=env1.reset()
	rews1 = 0	
	if episode > 1000:
		render=1
	if var<0.1:
		var=0

	for step in range(steps):
		if render:
			env1.render()
		#	env2.render()

		out1= agent_1.policy.predict((np.array([s1]),DUMMY_VALUE1,DUMMY_ACTION1))[0]
		mu_pred1,cov_pred1 =out1[0:a_dim1],out1[a_dim1:]
		cov_pred1 = np.diag(cov_pred1)
		action1= np.random.multivariate_normal(mu_pred1,cov_pred1,a_dim1)[0]
		#e1=np.random.normal(action1,var*np.ones(1,a_dim1),size=(1,a_dim1))[0]
		#e1=np.random.normal(action1,var,size=(1,a_dim1))[0] # need exploration
		a1 =np.clip(action1,-a_bound1,a_bound1 )
		s_1, reward1, done1, info1 = env1.step(a1)
		

		agent_1.memory.store(s1.ravel(),action1,s_1.ravel(),reward1,done1,out1,cov_pred1)# s, a, s_1, r, done1,musig
		rews1+=reward1


		if done1:
			s_1=env1.reset()
			
		s1 = s_1
	# updation
	print("env1[mcc] | "+str(episode)+" | "+str(rews1)+" | "+ str(var))
	print("_"*100)
	#print('training')
	agent_1.train(batch=64)



1.15.0
8
2
1.0
env1[mcc] | 1 | -3215.200451476637 | 1
____________________________________________________________________________________________________
env1[mcc] | 2 | -3619.5914049729645 | 1
____________________________________________________________________________________________________
env1[mcc] | 3 | -2902.015015072179 | 1
____________________________________________________________________________________________________
env1[mcc] | 4 | -3774.2830808598005 | 1
____________________________________________________________________________________________________
env1[mcc] | 5 | -2451.5194821209507 | 1
____________________________________________________________________________________________________
env1[mcc] | 6 | -2691.0950185312486 | 1
____________________________________________________________________________________________________
env1[mcc] | 7 | -2434.6388779517138 | 1
____________________________________________________________________________________________________
